In [42]:
import numpy as np
import pandas as pd

In [43]:
traindata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TRAIN.txt')#reading the data
testdata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TEST.txt')


/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
#CANTIDAD DE CADA VALOR. VEMOS QUE TENEMOS VALORES ENTEROS, CON x, QUE EMPIEZAN POR 0, REALES, ...
traindata['Socio_Demo_01'].value_counts()

09991     103393
09992      27266
09994      21070
0X301      15867
09993      15664
09511      13291
05220       8631
06110       8172
09512       7183
06120       5219
07121       4637
04500       4380
05120       3621
08432       2922
06300       2626
09210       2617
9992.0      2597
9991.0      2502
04111       2496
04309       2423
09602       2356
02230       1979
09700       1961
02240       1820
03613       1733
05000       1669
03611       1550
9512.0      1485
9994.0      1427
02121       1365
           ...  
2434.0         1
2640.0         1
2437.0         1
2630           1
8141           1
8132           1
2462.0         1
2463.0         1
2311           1
3134           1
3143.0         1
3152           1
9812           1
7622           1
3539           1
9542.0         1
3533           1
3531.0         1
3523           1
3403.0         1
7708           1
9223.0         1
3323           1
3317           1
2154           1
3209           1
2156           1
2157          

In [45]:
#Agrupa por columnas, saca la media de la columna target para cada grupo y ordena dichos grupos segun la media obtenida
def order_columns_by_target_mean(X,cols,target): 
    return X.groupby(cols)[target].mean().sort_values()

#Cantidad de valores faltantes de una o mas (o todas las) columnas
def n_missings(X,cols=''):
    if cols:
        return X[cols].isnull().sum() #solo algunas columnas
    else: 
        return X.isnull().sum() #todas las columnas

    
#GENERA LOS VALORES FALTANTES DE FORMA ALEATORIA PONDERADA POR LAS FRECUENCIAS RELATIVAS
#ES DECIR, CADA VALOR FALTANTE ES MAS PROBABLE QUE SE RELLENE CON LOS MAS FRECUENTES, 
#ASI SE MANTIENE LA DISTRIBUCION DE PROBABILIDAD ORIGINAL INTACTA    
def rellenar_faltantes_random_ponderado(X,col):
    #OBTENEMOS LA FRECUENCIA RELATIVA DE CADA VALOR
    val_count=X[col].value_counts()/X.shape[0]
    
    #A LA QUE TIENE MAS FRECUENCIA LE AÑADIMOS LO QUE FALTA PARA LLEGAR A 1(EN LA DIVISION SE PIERDEN DECIMALES Y FALTABA 0.0001 O ASI)
    val_count.iloc[0]+=1-val_count.values.sum()
    
    n_col_nulls=X[col].isnull().sum()
    return np.random.choice(val_count.index,n_col_nulls,p=val_count.values)

In [46]:
#CONVERTIMOS TODOS LOS VALORES EN NUMEROS 

grouped_by_socio=order_columns_by_target_mean(traindata,['Socio_Demo_01'],'Poder_Adquisitivo')
mapping={}
for i, index in enumerate(grouped_by_socio.index):
    mapping[index]=i
    
traindata['Socio_Demo_01']=traindata['Socio_Demo_01'].map(mapping)

#Se ha comprobado previamente que testdata no tiene valores que no esten incluidos en traindata
testdata['Socio_Demo_01']=testdata['Socio_Demo_01'].map(mapping)

In [47]:
#RELLENAMOS LOS VALORES FALTANTES DE Socio_Demo_01

traindata.loc[traindata['Socio_Demo_01'].isnull(),'Socio_Demo_01']=rellenar_faltantes_random_ponderado(traindata,'Socio_Demo_01')

testdata.loc[testdata['Socio_Demo_01'].isnull(),'Socio_Demo_01']=rellenar_faltantes_random_ponderado(testdata,'Socio_Demo_01')

#YA NO FALTAN
n_missings(traindata,'Socio_Demo_01'), n_missings(testdata,'Socio_Demo_01')

(0, 0)

In [48]:
#ESTABA TODAVIA EN FLOAT, LO PASAMOS A INT
traindata['Socio_Demo_01']=traindata['Socio_Demo_01'].astype('int')
testdata['Socio_Demo_01']=testdata['Socio_Demo_01'].astype('int')

traindata['Socio_Demo_01'].value_counts()

405    103568
61      27315
151     21110
294     15886
161     15683
179     13314
387      8646
467      8185
136      7197
366      5228
287      4645
557      4388
187      3628
593      2926
416      2629
140      2621
22       2602
200      2505
597      2500
578      2426
189      2358
833      1983
328      1963
802      1823
584      1735
282      1671
709      1552
74       1491
80       1428
731      1367
        ...  
915         1
409         1
427         1
425         1
795         1
157         1
794         1
26          1
293         1
18          1
386         1
129         1
878         1
12          1
117         1
42          1
887         1
14          1
888         1
889         1
378         1
714         1
891         1
457         1
381         1
16          1
382         1
528         1
896         1
0           1
Name: Socio_Demo_01, Length: 921, dtype: int64

In [49]:
#LA CORRELACION DE LA VARIABLE CON LA OBJETIVO (ESTO ES BASTANTE INUTIL DADO QUE ES UNA VARIABLE NOMINAL)
#LO COMENTO
traindata[['Socio_Demo_01','Poder_Adquisitivo']].corr()

,Socio_Demo_01,Poder_Adquisitivo
Socio_Demo_01,1.000000,0.196598
Poder_Adquisitivo,0.196598,1.000000


In [16]:
#GUARDO LOS DATOS LIMPIOS
traindata.to_csv('train.csv',sep=',')
testdata.to_csv('test.csv',sep=',')